この章ではpandasを用いたデータの入出力に話を絞る  
入出力は一般にいくつかの大きなカテゴリに分類される  
テキストファイルやもっと効率の良い形式のファイルをデータから読み込むパターン、データベースからのデータを読み込むパターン、Web APIなどのネットワーク上のソースデータを読み込むパターンなどがある

# 6.1  テキスト形式のデータの読み書き

pandasには、read_scvやread_tableなどを筆頭にテーブル形式のデータをデータフレームオブジェクトとして読み込む関数がたくさんある  

pandasのデータ読み込み関数

|関数|説明|
|:-|:-|
|read_scv|ファイルやURL、その他のファイル形のオブジェクトから、区切り文字で区切られたデータを読み込む。デフォルトの区切り文字はコンマ|
|read_table|ファイルやURL、その他のファイル系のオブジェクトから、区切り文字で区切られたデータを読み込む。デフォルトの区切り文字はタブ|
|read_fwf|列の幅が固定されている形式のデータ(区切り文字のないデータ)を読み込む|
|read_clipboard|read_tableの派生形で、クリップボードからデータを読み込む。ウェブページをテーブルに変換するのに便利|
|read_excel|EXCELのXLSやXLSXファイルからテーブル形式のデータを読み込む|
|read_hdf|pandasを用いて書き出したHDF5ファイルを読み込む|
|read_html|指定されたHTML文書に含まれているあらゆるテーブルを読み込む|
|read_json|JSONの文字列表現からデータを読み込む|
|read_msgpack|MessagePackバイナリ形式を用いて符号化されたpandasデータを読み込む|
|read_pickle|Pythonのpickle形式で書き出された任意のオブジェクトを読み込む|
|read_sas|SASシステム独自の保存形式のいずれかのバージョンで書き出されたSASデータセットを読み込む|
|read_sql|SQLクエリをSQLAlchemyを用いて発行した結果をpandasデータをフレームとして読み込む|
|read_state|Stateファイル形式からデータセットを読み込む|
|read_feather|Featherバイナリファイル形式を読み込む|

これらの関数はテキストデータをデータフレームに変換するためのもので、そのオプション引数はいくつかのカテゴリに分類できる
- インデックス化  
これを用いると、1つまたは複数の列を戻り値として得られるデータフレームのインデックスとして扱うことができる  
また、列名をファイルから読み込むか、ユーザーに与えるか、あるいは何も与えないか決めることができる  
- 型推論とデータ変換  
ユーザーの定義した値の変換や、どのデータを欠損値として扱うかを決めるリストのカスタマイズはこれに含まれる  
- 日付の読み込み  
複数の列に分散している日付情報を1つの列にまとめ上げる結合機能はこれに含まれる  
- イテレーション  
巨大ファイルのデータの塊に対するイテレーションのサポート  
- 整っていないデータの問題  
一部の行やフッター、コメントの読み飛ばしや、3桁ごとにコンマで区切られている数値データの読み込みのような小さな問題

現実世界のデータは非常に取り扱いにくいことがあるため、いくつかのデータの読み込み関数はオプションが非常に複雑になってしまっている  
オンラインのpandasのドキュメントには、それぞれのオプションでどのように動きが変わるのかという例がたくさん載っている  
入力データ形式によっては各列のデータ型の情報を含んでいないことがあるため、読み込み関数のいくつかでは、型推論が行われる  
どの列が数値か、真偽値か、文字列かを読み込むときに明示的に指定する必要は必ずしもない  
HDF5やFeather、msgpackなどのデータ形式には、内部にデータの型情報が含まれている  
しかし、日付などの特別な型を扱うときは、余計な手間が必要になることがある

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r".\download_file\examples\ex1.csv")
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


csvファイルを読み込むには、pandasのread_csvにcsvファイル名を渡すことでデータフレームにして読み込むことができる  

In [3]:
pd.read_table(r".\download_file\examples\ex1.csv", sep=",")

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


pandasのread_table関数でもcsvファイルを読み込むことができるが、区切り文字のコンマをキーワード引数のsepに渡す必要がある

In [4]:
pd.read_csv(r".\download_file\examples\ex2.csv")

,1,2,3,4,hello
0,5,6,7,8,world
1,9,10,11,12,foo


In [5]:
pd.read_csv(r".\download_file\examples\ex2.csv", header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
pd.read_csv(r".\download_file\examples\ex2.csv", names=["a", "b", "c", "d", "message"])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


csvファイルにヘッダ行がない場合は、自動で最初の行が列名にならないようにする必要がある  
そのためには、キーワード引数のheaderにNoneを渡して最初の行が列名にならないようにするか、  
キーワード引数のnamesに列名のリストを渡して列名を付けることで回避できる

In [7]:
names = ["a", "b", "c", "d", "message"]
pd.read_csv(r".\download_file\examples\ex2.csv", names=names, index_col="message") # messageの部分は4でも可

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


取得したデータフレームの特定の列を各行のインデックスとして使いたい場合は、キーワード引数のindex_colに列名もしくは列のインデックス値を渡すことでできる

In [8]:
pd.read_csv(r".\download_file\examples\csv_mindex.csv")

,key1,key2,value1,value2
0,one,a,1,2
1,one,b,3,4
2,one,c,5,6
3,one,d,7,8
4,two,a,9,10
5,two,b,11,12
6,two,c,13,14
7,two,d,15,16


In [9]:
pd.read_csv(r".\download_file\examples\csv_mindex.csv", index_col=["key1", "key2"])

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

行のインデックスとして複数の列で構成される階層型インデックスを作りたい場合は、index_colに列名のリストを渡すことでできる

In [10]:
list(open(r".\download_file\examples\ex3.txt"))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [11]:
pd.read_table(r".\download_file\examples\ex3.txt", sep="\s+")

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


テーブル形式のデータの中には決まった区切り文字のない場合がある  
上のようにテキストファイルで、フィールドを分割するのに空白文字を使っているような場合になる  
テーブル形式であればread_tableを使うことができ、キーワード引数のsepには正規表現を渡して区切り文字を指定できる  
上の場合は空白文字があればフィールドを分けるようにしている  
また、最初の行の列数が他のデータの列数よりも1つ少なくなっていることから、自動的に1つ目の行が列のインデックス名になっていると推測してくれている

In [12]:
pd.read_csv(r"download_file\examples\ex4.csv")

,,,,# hey!
a,b,c,d,message
# just wanted to make things more difficult for you,NaN,NaN,NaN,NaN
# who reads CSV files with computers,anyway?,NaN,NaN,NaN
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [13]:
pd.read_csv(r".\download_file\examples\ex4.csv", skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


csvファイルなどで余計な行を飛ばして読み込みたいときには、キーワード引数のskiprowsに飛ばしたい行のインデックスのリストを渡すことでできる

In [14]:
result = pd.read_csv(r".\download_file\examples\ex5.csv")
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [15]:
result.isnull()

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


欠損値の扱いは、ファイルを読み込むうえで需要な部分であり、しばしばファイルごとに異なる部分になる  
欠損値は通常、値が存在しないか、何らかの標識となる値で印をつけられている  
pandasの場合は、デフォルトではNAやNULLなどの一般によく使われる標識を使う

In [16]:
pd.read_csv(r".\download_file\examples\ex5.csv", na_values=["foo"])

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


データを読み込む際に、欠損値として扱う要素を決めることができ、キーワード引数のna_valuesに欠損値として扱いた要素のリストや文字列を渡すことでできる  

In [17]:
sentinels = {"message": ["foo", "NA"], "something": ["two"]}
pd.read_csv(r".\download_file\examples\ex5.csv", na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


列ごとに欠損値にする要素を決める場合は、na_valuesに列名がキーで指定する要素を値とした辞書を渡すことでできる

read_csvとread_tableによく与える引数

|引数|説明|
|:-|:-|
|path|ファイルシステム上の位置やURL、その他のファイル形式のオブジェクトを示す文字列|
|sep, delimiter|各行をフィールドに分割するのに用いる文字列あるいは正規表現|
|header|列名として使う行の番号。デフォルトでは0(最初の行)。ヘッダ行がない場合にはNoneを指定する|
|index_cols|戻り値として得られるオブジェクトにおいて、行のインデックスとして使われる列の番号か名前。単一の名前・番号か、階層型インデックスの場合は名前・番号のリスト|
|names|戻り値として得られるオブジェクトの列名のリスト。header=Noneと共に使用する|
|skiprows|ファイルの先頭で無視する行数か、読み飛ばす番号|
|na_values|欠損値に置き換える一連の値|
|comment|この引数に指定した文字または文字列以降を各行からコメントとして切り離す|
|parse_dates|データを日時として読み込もうとする。デフォルトではFalseで、Trueの場合はすべての列で読み込もうとする。すべての列に適用したくない場合は、読み込む列の番号か名前を指定する。リストのリストで指定すると、それらの複数の列を結合して日付として読み込む|
|keep_date_col|複数の列を結合して日付として読み込む場合に、結合に用いられた列を残す。デフォルトではFalse|
|converters|列番号を表す名前を、関数にマッピングする。辞書で指定し、{"foo": func}のようにすることでfoo列のすべての要素に関数funcを適用できる|
|dayfirst|複数の解釈ができる可能性のある日付を読み込む際に、ヨーロッパで標準的な日付表記形式(31/12/2020)の形式として取り扱う。デフォルトではFalse|
|dare_parser|日付を読み込むのに用いる関数|
|nrows|ファイル先頭で読み込む行数|
|iterator|ファイルを部分的に読み込むためのTextFileReaderオブジェクトを戻り値とする|
|chunksize|イテレーションに用いるファイル内のブロックのサイズ|
|skip_footer|ファイル末尾で無視する行数|
|verbose|パーサー出力するさまざまな情報を表示する。例えば非数値の列に見つかった欠損値の数などを表示する|
|encoding|Unicodeとして用いる文字コードを指定できる|
|squeeze|読み込まれたデータに1つの列しか含まれていない場合、シリーズを戻り値とする|
|thousands|3桁区切りのセパレータ。例えば","や"."|

## 6.1.1  テキストファイルを少しずつ読み込む

非常に大きなファイルを処理する場合や、巨大ファイルを正しく処理するためにどの引数を与えるべきなのか確かめるために、1つのファイルのごく一部だけを読み込んだり、ファイル内の小さなブロックごとに処理を繰り返したりしたい

In [18]:
pd.options.display.max_rows = 10

上のようにpandasのoptions.display.max_rowsに10を渡して、表示する行数を少なく設定しておく(デフォルトは60)

In [19]:
result = pd.read_csv(r".\download_file\examples\ex6.csv")
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [20]:
pd.read_csv(r".\download_file\examples\ex6.csv", nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


上のような大きなファイルの最初の数行だけ読み込みたいときは、キーワード引数のnrowsに行数を指定することで指定した行数分だけ読み込むことができる

In [21]:
chunker = pd.read_csv(r".\download_file\examples\ex6.csv", chunksize=1000)
chunker
next(chunker)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
995,2.311896,-0.417070,-1.409599,-0.515821,M
996,-0.479893,-0.650419,0.745152,-0.646038,H
997,0.523331,0.787112,0.486066,1.093156,D
998,-0.362559,0.598894,-1.843201,0.887292,W


In [22]:
chunker.get_chunk(100)

,one,two,three,four,key
1000,0.467976,-0.038649,-0.295344,-1.824726,T
1001,-0.358893,1.404453,0.704965,-0.200638,J
1002,-0.501840,0.659254,-0.421691,-0.057688,R
1003,0.204886,1.074134,1.388361,-0.982404,S
1004,0.354628,-0.133116,0.283763,-0.837063,B
...,...,...,...,...,...
1095,1.106521,0.098153,0.789793,1.192693,8
1096,-0.540543,1.782569,0.051931,0.463868,Z
1097,-0.101980,0.981720,1.106990,-1.752269,L
1098,0.632107,-0.761419,1.427930,-0.046928,E


In [23]:
chunker = pd.read_csv(r".\download_file\examples\ex6.csv", chunksize=1000)
tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece["key"].value_counts(), fill_value=0)
tot = tot.sort_values(ascending=False)
tot[:10]

C:\Users\kumak\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

ファイルを少しずつ読み込みたい場合はキーワード引数のchunksizeで行数を指定することでできる  
そうするとデータフレームオブジェクトでなくTextFileReaderオブジェクトが返され、イテレータのように扱いうことで少しずつ読み込むことができる  
上では、からのシリーズを作り、TextFileReaderオブジェクトから1000行ずつに分けてデータを取得し、key列の要素の値ごとに行の数値を足していき、降順で上の10個の要素のみを表示している

## 6.1.2  テキスト形式でのデータの書き出し

データフレームのデータを、区切り文字で区切られた形式でのエクスポート(書き出し)もできる  

In [24]:
data = pd.read_csv(r".\download_file\examples\ex5.csv")
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [25]:
data.to_csv(r".\pydata\out.csv")

書き出しはデータフレームオブジェクトのto_csvメソッドを使うことでCSVファイルとしてデータを書き出すことができる

In [26]:
import sys
data.to_csv(sys.stdout, sep="|")

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


to_csvメソッドの区切り文字はデフォルトではコンマ区切りだが、他の区切り文字も使うことができ、キーワード引数のsepに区切り文字を与えることでできる  
上ではファイルに保存する代わりにsysモジュールのstdoutを使って単に出力している

In [27]:
data.to_csv(sys.stdout, na_rep="NULL")

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


欠損値は空文字として保存されるが、キーワード引数のna_repに欠損値の代わりに入れたい文字列を渡すことで、欠損値を別の文字列で埋めることができる

In [28]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


デフォルトでは行と列の両方のラベルも書き出されるが、キーワード引数のindexとheaderにFalseを渡すことで、それぞれ行と列のラベルを書き出さないように設定できる

In [29]:
data.to_csv(sys.stdout, index=False, columns=["a", "b", "c"])

a,b,c
1,2,3.0
5,6,
9,10,11.0


キーワード引数のcolumnsに列名のリストを渡すことで、一部の列だけを指定して書き出すこともできる

In [30]:
dates = pd.date_range("1/1/2000", periods=7)
dates

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07'],
              dtype='datetime64[ns]', freq='D')

In [31]:
import numpy as np
ts = pd.Series(np.arange(7), index=dates)
ts

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
Freq: D, dtype: int32

In [32]:
ts.to_csv(sys.stdout, header=False)

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


シリーズもto_csvメソッドで書き出すことができる

## 6.1.3  区切り文字で区切られた形式を操作する

テーブル形式のデータの大半は、pandasのread_tableのような関数を用いて読み込むことができる  
しかし、データの内容次第では多少のデータ修正が必要になる場合がある

In [33]:
import csv
f = open(r".\download_file\examples\ex7.csv")
reader = csv.reader(f)
for line in reader:
    print(line)
f.close()

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [34]:
f = open(r".\download_file\examples\ex7.csv")
reader = csv.reader(f)
a, b, c = reader

In [35]:
a

['a', 'b', 'c']

In [36]:
b

['1', '2', '3']

In [37]:
c

['1', '2', '3']

In [38]:
f.close()

区切り文字が1文字のどのようなファイルでも組み込みのcsvモジュールを使うことができる  
csvモジュールを使い、csvファイルを読み込むにはcsvモジュールのreader関数にcsvファイルのファイルオブジェクトを渡すことでできる、readerオブジェクトを生成することでできる  
readerオブジェクトに対して繰り返し処理を行うことで、全ての要素に引用符のついた各行ごとのリストとして返される

In [39]:
with open(r".\download_file\examples\ex7.csv") as f:
    lines = list(csv.reader(f))

In [40]:
header, values = lines[0], lines[1:]

In [41]:
header

['a', 'b', 'c']

In [42]:
values

[['1', '2', '3'], ['1', '2', '3']]

取得したcsvファイルを必要な形の形式にするためにはいろいろなやり方があるが、  
上ではリスト(全体)のリスト(各行)として、最初のリストをヘッダー行、2番目以降のリストをデータ行として代入している

In [43]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [44]:
pd.DataFrame(data_dict)

,a,b,c
0,1,2,3
1,1,2,3


上ではディクショナリ内包表記の中でzip関数を使ってヘッダー行とデータ行を順番に取り出して、ヘッダー行の要素をキー、データ行の要素を値とした辞書を作成している

In [45]:
class my_dialect(csv.Dialect):
    lineterminator = "\n"
    delimiter = ";"
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

In [46]:
with open(r".\download_file\examples\ex5.csv") as f:
    reader = csv.reader(f, dialect=my_dialect)
    for line in reader:
        print(line)

['something,a,b,c,d,message']
['one,1,2,3,4,NA']
['two,5,6,,8,world']
['three,9,10,11,12,foo']


csvファイルにはたくさんの異なる方言があり、区切り文字、文字列を引用符でくくるルール、行終端記号などによって異なる  
方言の設定はcsv.Dialectから新しいサブクラスを作り、値を設定することでできる

In [47]:
with open(r".\download_file\examples\ex5.csv") as f:
    reader = csv.reader(f, delimiter="|")
    for line in reader:
        print(line)

['something,a,b,c,d,message']
['one,1,2,3,4,NA']
['two,5,6,,8,world']
['three,9,10,11,12,foo']


新たなサブクラスを定義するのではなく、csvの方言のパラメータをキーワード引数として文字列を渡すことでも設定できる  

CSVの方言に使えるオプション

|引数|説明|
|:-|:-|
|delimiter|フィールドに分割するための1文字の文字列。デフォルトは","|
|lineterminator|書き出し用の行終端記号。デフォルトは"\r\n"。readerで読み込む際にはこの指定は無視され、クロスプラットフォームの行終端記号が認識される|
|quotechar|区切り文字などの特殊文字を含んだフィールドのための引用符。デフォルトは'"'|
|quoting|引用符でくくるルール。csv.QUOTE_ALL(全てのフィールドを引用符でくくる)、csv.QUOTE_MINIMAL(区切り文字のような特殊文字を含んだフィールドだけくくる)、csv.QUOTE_NONNUMERIC(数字以外をくくる)、csv.QUOTE_NON(くくらない)を選択できる。デフォルトはcsv.MINIMAL|
|skipinitialspase|各区切り文字の直後の空白を無視するか。デフォルトはFalse|
|doublequote|フィールド内の引用符をどう取り扱うか。Trueの場合、二重引用符になる|
|escapechar|上記のquoteingにcsv.QUOTE_NONEが設定されている場合に、区切り文字をエスケープするための文字列。デフォルトでは無効|

In [48]:
with open(r".\pydata\mydata.csv", "w") as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(("one", "two", "three"))
    writer.writerow(("1", "2", "3"))
    writer.writerow(("4", "5", "6"))
    writer.writerow(("7", "8", "9"))

区切り文字で区切られたファイルとして手動で書き出すには、csv.writer関数でwriterオブジェクトを生成することでできる  
このwriterオブジェクトを作成する際に、作成したサブクラスを使ったり、キーワード引数を指定してcsvの方言を設定することができる

## 6.1.4  JSONデータ

JSONはウェブブラウザと他のアプリケーションの間でHTTPリクエストでデータをやり取りするうえで、標準的なデータ形式のひとつである  
CSVのようなテーブル形式のテキストデータと比べると、JSONははるかに自由度が高くフリーフォームなデータ形式になる

In [49]:
# JSON形式のデータ例

obj = """
{"name":"Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

JSONはnull値がnullである点や、その他の差異(リストの最終要素の後ろにコンマを入れられないなど)を除けば、ほぼPythonのコードそのものである  
基本型はオブジェクト(ディクショナリ)、配列(リスト)、文字列、数値、真偽値、nullで構成されている  
オブジェクトのキーはすべて文字列でなければならない  
JSONデータを読み書きするためのPythonライブラリはいくつかあり、ここでは、Python標準ライブラリのjsonを使う  

In [50]:
import json

In [51]:
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

jsonモジュールのloads関数にJSON形式の文字列を渡すことでPython形式のオブジェクトに変換することができる

In [52]:
asjson = json.dumps(result)
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

jsonモジュールのdumps関数にPythonオブジェクトを渡すことでJSON形式の文字列に変換することができる

In [53]:
pd.DataFrame(result["siblings"], columns=["name", "age"])

,name,age
0,Scott,30
1,Katie,38


JSONオブジェクトやオブジェクトのリストを、どのようにしてデータフレームやその他の分析に適したデータ構造に変換する方法はいくつかあり、  
便利なのは、上のようにディクショナリ(JSONオブジェクトをPython形式に変換したもの)のリスト部分をpandasのDataFrame関数に渡して、一部のデータフィールドを選択するという方法になる

In [54]:
with open(r".\download_file\examples\example.json") as f:
    print(json.load(f))

[{'a': 1, 'b': 2, 'c': 3}, {'a': 4, 'b': 5, 'c': 6}, {'a': 7, 'b': 8, 'c': 9}]


In [55]:
data = pd.read_json(r".\download_file\examples\example.json")
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


別の方法として上のように、pandasモジュールのread_json関数を使う方法がある  
read_json関数にファイルのパスを渡すと、特定の形のデータが連なったJSONデータセットをシリーズやデータフレームへと自動的に変換することができる  
read_json関数はデフォルトだと、JSON配列内の各オブジェクトがテーブルの行になると仮定される

In [56]:
data.to_json(sys.stdout)

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}

In [57]:
data.to_json(sys.stdout, orient="records")

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]

pandasのオブジェクトをJSON形式でエクスポートする必要がある場合、シリーズやデータフレームのto_jsonメソッドを使うことで書き出しができる

## 6.1.5  XMLとHTML : ウェブスクレイピング

PythonにはHTML/XML形式のデータの読み書き用のライブラリがたくさんあり、代表的なものとしてlxmlやBeautifulSoup、html5libなどがある  
lxmlは高速に動くが、規格に従っていないHTMLファイルやXMLファイルを取り扱うにはBeautifulSoupやhtml5libのほうが便利になる  
pandasには組み込み関数のread_html関数がありlxmlやBeautifulSoupなどのライブラリを用いて自動的にHTMLファイルをパースし、ファイル内に含まれているテーブルをデータフレームオブジェクトとして取り出してくれる  

例としてアメリカの政府機関、連邦預金保険会社(FDIC)から現行の破綻を説明するHTMLファイルを使う

In [58]:
tables = pd.read_html(r".\download_file\examples/fdic_failed_bank_list.html")
tables

[                             Bank Name             City  ST   CERT  \
 0                          Allied Bank         Mulberry  AR     91   
 1         The Woodbury Banking Company         Woodbury  GA  11297   
 2               First CornerStone Bank  King of Prussia  PA  35312   
 3                   Trust Company Bank          Memphis  TN   9956   
 4           North Milwaukee State Bank        Milwaukee  WI  20364   
 ..                                 ...              ...  ..    ...   
 542                 Superior Bank, FSB         Hinsdale  IL  32646   
 543                Malta National Bank            Malta  OH   6629   
 544    First Alliance Bank & Trust Co.       Manchester  NH  34264   
 545  National State Bank of Metropolis       Metropolis  IL   3815   
 546                   Bank of Honolulu         Honolulu  HI  21029   
 
                    Acquiring Institution        Closing Date  \
 0                           Today's Bank  September 23, 2016   
 1              

In [59]:
len(tables)

1

pandasのread_html関数にHTMLファイルのパス名を渡して、データフレームオブジェクトのリストを取得する  
read_html関数にはオプションがたくさんあるが、デフォルトでは<table\>タグで囲まれたテーブル形式のデータをすべて探し出し、パースしようとする

In [60]:
failures = tables[0]

In [61]:
failures

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"
...,...,...,...,...,...,...,...
542,"Superior Bank, FSB",Hinsdale,IL,32646,"Superior Federal, FSB","July 27, 2001","August 19, 2014"
543,Malta National Bank,Malta,OH,6629,North Valley Bank,"May 3, 2001","November 18, 2002"
544,First Alliance Bank & Trust Co.,Manchester,NH,34264,Southern New Hampshire Bank & Trust,"February 2, 2001","February 18, 2003"
545,National State Bank of Metropolis,Metropolis,IL,3815,Banterra Bank of Marion,"December 14, 2000","March 17, 2005"


例のリストからインデックスを指定してデータフレームオブジェクトを取り出すと、上のようになっている

In [62]:
close_timestamps = pd.to_datetime(failures["Closing Date"])
close_timestamps

0     2016-09-23
1     2016-08-19
2     2016-05-06
3     2016-04-29
4     2016-03-11
         ...    
542   2001-07-27
543   2001-05-03
544   2001-02-02
545   2000-12-14
546   2000-10-13
Name: Closing Date, Length: 547, dtype: datetime64[ns]

In [63]:
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

取得したデータフレームオブジェクトを使って年ごとの銀行破綻件数を算出すると上のようになる

### 6.1.5.1  lxml.objectifyを使ったXMLの読み込み

XMLは、もう一つのよく使われる構造化されたデータ形式で、メタデータを用いたデータの階層化やネスト構造をサポートしている  
XMLとHTMLは構造的には似ているが、XMLのほうが汎用的で、だまざまなデータに使える  
以下では、XML形式のデータからlxmlを使ってデータを読み出す方法を、例を用いて説明する  
ここではXMLファイルに含まれている業績データを見ていく

In [64]:
from lxml import objectify

In [65]:
path = r".\download_file\datasets\mta_perf\Performance_MNR.xml"

In [66]:
parsed = objectify.parse(open(path))
root = parsed.getroot()

毎月のデータが含まれているXMLファイルを使ってファイルオブジェクトを作成し、lxml.objectifyモジュールのparse関数に渡してXMLファイルを読み込み、getrootメソッドでファイルのルートノードへの参照を取得している

In [67]:
root.INDICATOR

<Element INDICATOR at 0x1d617e6bb08>

In [68]:
data = []

In [69]:
skip_fields = ["PARENT_SEQ", "INDICATOR_SEQ", "DESIRED_CHANGE", "DECIMAL_PLACES"]

In [70]:
for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [71]:
data

[{'AGENCY_NAME': 'Metro-North Railroad',
  'INDICATOR_NAME': 'On-Time Performance (West of Hudson)',
  'DESCRIPTION': 'Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. West of Hudson services include the Pascack Valley and Port Jervis lines. Metro-North Railroad contracts with New Jersey Transit to operate service on these lines.\n',
  'PERIOD_YEAR': 2008,
  'PERIOD_MONTH': 1,
  'CATEGORY': 'Service Indicators',
  'FREQUENCY': 'M',
  'INDICATOR_UNIT': '%',
  'YTD_TARGET': 95.0,
  'YTD_ACTUAL': 96.9,
  'MONTHLY_TARGET': 95.0,
  'MONTHLY_ACTUAL': 96.9},
 {'AGENCY_NAME': 'Metro-North Railroad',
  'INDICATOR_NAME': 'On-Time Performance (West of Hudson)',
  'DESCRIPTION': 'Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. West of Hudson services include the Pascack Valley and Port Jervis lines. Metro-North Railroad contracts with New Jersey Transit to operate 

root.INDICATORとすると、各<INDICATOR\>XML要素を生成するジェネレータが返される  
これを使って、各レコードについて、タグ名とそれに対応するデータの値を持つディクショナリを作成する

In [72]:
perf = pd.DataFrame(data)
perf

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95,96.9,95,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95,96,95,95
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95,96.3,95,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95,96.8,95,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95,96.6,95,95.8
...,...,...,...,...,...,...,...,...,...,...,...,...
643,Metro-North Railroad,Escalator Availability,Percent of the time that escalators are operat...,2011,8,Service Indicators,M,%,97,,97,
644,Metro-North Railroad,Escalator Availability,Percent of the time that escalators are operat...,2011,9,Service Indicators,M,%,97,,97,
645,Metro-North Railroad,Escalator Availability,Percent of the time that escalators are operat...,2011,10,Service Indicators,M,%,97,,97,
646,Metro-North Railroad,Escalator Availability,Percent of the time that escalators are operat...,2011,11,Service Indicators,M,%,97,,97,


最後に、作成しディクショナリをデータフレームに変換する

XMLデータは、各タグにもメタデータを付けられるために、この例よりもはるかに複雑なことがある  
HTMLのリンクタグを考えると、リンクタグは妥当なXMLでもある

In [73]:
from io import StringIO

In [74]:
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

In [75]:
root.get("href")

'http://www.google.com'

In [76]:
root.text

'Google'

上のようにタグ内のどんなフィールドにも、リンクテキストにもアクセスできる

# 6.2  バイナリデータ形式

バイナリ形式で効率よくデータを書き出す(シリアライズ)最も簡単な方法のひとつは、組み込み関数のpickleによるシリアライズを使うことになる  

In [77]:
frame = pd.read_csv(r".\download_file\examples\ex1.csv")
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [78]:
frame.to_pickle(r".\pydata\frame_pickle")

pandasオブジェクトはすべて、データをpickle形式でディスクに書き出すto_pickleメソッドを持っていて、ファイルのパスを渡すことでシリアライズできる

In [79]:
pd.read_pickle(r".\pydata\frame_pickle")

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


pickle化されてファイルに書き出されたオブジェクトは、組み込みのpickleモジュールを用いて直接読み込める  
もっと便利な方法として上のように、pandasのread_pickleを使うことで読み込むこともできる

pickleの他にも、HDF5とMessagePackという2つのバイナリデータ形式のサポートが含まれている  
pandasやNumPyのデータとして使えるその他の保存形式として以下のようなものがある  
- bcolz  
Blosc圧縮ライブラリをベースとした、圧縮可能な列指向なのバイナリ形式
- Feather  
著者がRプログラミングコミュニティのHadley Wickhamとともに設計した、言語横断的な列指向のファイル形式。Featherでは、Apache Arrowの列指向のメモリ形式を使っている

## 6.2.1  HDF5形式の使用

HDF5は大量の化学的な配列データを保存するための、評判の良いファイル形式になる  
HDF5はCで書かれたライブラリだが、Java、Julia、MATLAB、Pythonなど多数の言語向けインタフェースがある  
HDFは階層型データ形式という意味がある  
各HDF5ファイルには、複数のデータセットや、それらのデータセットの情報を収めるメタデータの保存が可能である  
もっと単純なデータ形式と比べると、HDF5は様々な圧縮モードでのオンザフライ(中間ファイルを出力しない)の圧縮をサポートしており、繰り返しパターンを持ったデータを効率よく保存できるようになっている  
HDF5では巨大な配列のごく一部を効率よく読み書きできるので、メモリに収まらない非常に巨大なデータセットに使うに適している

PyTablesライブラリとh5pyライブラリのどちらかを使えば、HDF5ファイルへの直接アクセスが可能になる  
しかし、pandasではシリーズやデータフレームのオブジェクトを簡単に書きだすための高レベルなインタフェースが提供されている  
HDFStoreクラスはディクショナリのように扱えるインタフェースで、低レベルな詳細部分は内部的に処理してくれる

In [80]:
frame = pd.DataFrame({"a": np.random.randn(100)})

In [81]:
store = pd.HDFStore(r".\pydata\mydata.h5")

In [82]:
store["obj1"] = frame

In [83]:
store["obj1_col"] = frame["a"]

In [84]:
print(store.info())

<class 'pandas.io.pytables.HDFStore'>
File path: .\pydata\mydata.h5
/obj1                frame        (shape->[100,1])
/obj1_col            series       (shape->[100])  


上のようにHDFStoreクラスにファイル名を渡すことで.h5ファイルを作成することができ、返されたHDFStoreオブジェクトをディクショナリのように使うことでオブジェクトを保存できる  
infoメソッドを呼ぶことで内部情報を返してくれる

In [85]:
store["obj1"]

,a
0,1.740677
1,2.429939
2,-0.171708
3,0.398418
4,0.280968
...,...
95,-0.507262
96,1.690077
97,-0.093753
98,-1.014135


HDFStoreオブジェクトもディクショナリのようなAPIを用いて取り出すことができる

In [86]:
store.put("obj2", frame, format="table")

In [87]:
store["obj2"]

,a
0,1.740677
1,2.429939
2,-0.171708
3,0.398418
4,0.280968
...,...
95,-0.507262
96,1.690077
97,-0.093753
98,-1.014135


HDFStoreオブジェクトにputメソッドを使うことでも書き出しができる  
第一引数にキーにあたる文字列、第二引数に保存するオブジェクトを渡す  
キーワード引数のformatには"fixed"と"table"を渡すことができ、通常tableのほうが低速だが、特殊な文法を用いたクエリ操作をサポートしているという利点がある  

In [88]:
store.close()

In [89]:
frame.to_hdf(r".\pydata\mydata.h5", "obj3", format="table")

In [90]:
pd.read_hdf(r".\pydata\mydata.h5", "obj3", where=["index < 5"])

,a
0,1.740677
1,2.429939
2,-0.171708
3,0.398418
4,0.280968


HDFStoreオブジェクトを閉じた後に、ファイルに書き出したいとき、pandasオブジェクトのto_hdfメソッドを使って、ファイル名と保存する名前を渡すことで書き出すことができる  
逆に読み込む場合は、pandasのread_hdf関数にファイル名と取り出したオブジェクトの名前を渡すことで取り出すことができる  
その際にキーワード引数のwhereにオブジェクトの範囲を指定する情報を渡すことができ、一部のみを取り出すことができる  

## 6.2.2  Microsoft Excelファイルの読み込み

pandasでは、Excel2003のファイルに保存されたテーブルデータの読み込みもできる  
読み込みは、ExcelFileクラスかpandasのread_excel関数のどちらかを使って行うことができる  
これらのツールは内部で、読み込みにはxlrd、書き込みにはxlwtやopenpyxlというパッケージを使用しているので、使う場合にはインストールしておく必要がある  

In [91]:
xlsx = pd.ExcelFile(r".\download_file\examples\ex1.xlsx")

ExcelFileを使うにはExcelFileオブジェクトを生成する必要があり、pandasのExcelFile関数にファイルパスを渡すことでできる  

In [92]:
pd.read_excel(xlsx, "Sheet1")

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


pandasのread_excel関数にExcelFileオブジェクトと、読み込みたいシート名を渡すことで読み込むことができる 

In [93]:
pd.read_excel(r".\download_file\examples\ex1.xlsx", "Sheet1")

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


複数のシートを読み込む場合にはExcelFileオブジェクトをread_excel関数に渡した方が高速に読み込めるが、1つのシートだけを読み込むのならファイル名とシート名を渡すだけでも読み込むことができる

In [94]:
writer = pd.ExcelWriter(r".\pydata\ex1.xlsx")

In [95]:
frame.to_excel(writer, "Sheet1")

In [96]:
writer.save()

pandasのデータをExcel形式で書き出すには、pandasのExcelWriter関数にファイル名を渡してExcelWriterオブジェクトを作成して、  
pandasのオブジェクトのto_excelメソッドに作成したExcelWriterオブジェクトと保存したいシート名を渡すことで書き出しができる  
最後にExcelWriterオブジェクトのsaveメソッドで内容を保存することで完了する

In [97]:
frame.to_excel(r".\pydata\ex2.xlsx")

ExcelWriterオブジェクトを使わずにto_excelメソッドに直接ファイル名を渡すことでも書き出すことができる

# 6.3  Web APIを用いたデータの取得

多くのウェブサイトには、JSONなどの形式でデータフィードを提供している公開APIがあり、このようなAPIにPythonからアクセスするには多くの方法がある  
その中でも簡単に使える方法がrequestsパッケージを使う方法になる  
以下では例として、GitHub上のpandasのGitHub Issuesから最新の課題30個を取得するためにrequestsライブラリをアドオンとして用いて、HTTPリクエストGETを送信してみる

In [98]:
import requests

In [99]:
url = "http://api.github.com/repos/pandas-dev/pandas/issues"

In [100]:
resp = requests.get(url)
resp

<Response [200]>

requestsにはgetメソッドがあり、URLを渡すことでHTTPリクエストを送ることができ、成功すると様々な情報の入ったresponseオブジェクトを返す

In [101]:
data = resp.json()

In [102]:
data[0]["title"]

'CLN: Format doc code blocks'

responseオブジェクトのjsonメソッドを使うことで、JSONから読み出されたネイティブのPythonオブジェクトを含むディクショナリが戻される

In [103]:
issues = pd.DataFrame(data, columns=["number", "title", "labels", "state"])
issues

,number,title,labels,state
0,36700,CLN: Format doc code blocks,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
1,36699,CLN: Remove unused fixtures,"[{'id': 211029535, 'node_id': 'MDU6TGFiZWwyMTE...",open
2,36698,Categorical in GroupBy with aggregations raise...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
3,36697,DEPR: is_all_dates,[],open
4,36696,TYP: mostly datetimelike,[],open
...,...,...,...,...
25,36656,ASV: used integer ndarray as indexer for Serie...,[],open
26,36655,BUG: DataFrame.pivot drops column level names ...,[],open
27,36654,CLN: dont special-case DatetimeArray indexing,[],open
28,36653,CLN: de-duplicate IntervalArray validators,[],open


data内の各要素は、GitHub上のIssuesのページに含まれているディクショナリで、dataから直接データフレームをコンストラクタに渡して、取り出したフィールドを指定することでデータフレームを得られる

# 6.4  データベースからのデータの取得

ビジネスシーンにおいて、大半のデータはSQLベースのリレーショナルデータベース(SQL Server、PostgreSQL、MySQLなど)が広く使われている  
それ以外にも多くの代替となるデータベースもあり、どのデータベースを使うかは、アプリの要求するパフォーマンス、データ安全性、スケーラビリティによって決められる  
SQLからデータを読み込んでデータフレームに入れるのはかなり直観的な処理で、pandasにはこの処理を簡単にしてくれる関数がいくつかある  
まずは読み込みの例を実行するために、Pythonの組み込みライブラリのsqlite3を用いてSQLiteデータベースを作成する

In [104]:
import sqlite3

In [105]:
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
 );"""

In [106]:
con = sqlite3.connect(r".\pydata\mydata.sqlite")

In [107]:
con.execute(query)

In [108]:
con.commit()

以上でデータベースの作成ができた

In [109]:
data = [("Atlanta", "Georgia", 1.25, 6),
        ("Tallahassee", "Florida", 2.6, 3),
        ("Sacramento", "California", 1.7, 5)]

In [110]:
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

In [111]:
con.executemany(stmt, data)

In [112]:
con.commit()

以上でデータの挿入ができたので、以下で読み込んでいく

In [113]:
cursor = con.execute("select * from test")

In [114]:
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

PythonのSQLドライバの大半はテーブルのデータを選択したときにタプルのリストを返す

In [115]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [116]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


このタプルのリストをそのままデータフレームにしてもよいが、列名も一緒に渡したい  
列名の値はカーソルのdescription属性に含まれているので、列名の設定でリスト内包表記を用いて設定している

これはかなり面倒な処理で、データベースをクエリするときに毎回繰り返すのは嫌になる  
SQLAlchemyプロジェクトはSQLデータベース間によくある差異の多くを抽象化して取り除いてくれる人気のあるPython用のツールキットを提供している  
このSQLAlchemyの汎用的なデータベースを接続を通じて簡単にデータを読み出せる  
pandasにはread_sqlという関数があり、以下では先程と同じSQLiteデータベースにSQLAlchemyを用いて接続し、先ほど作成したテーブルからデータを読み込んでみる

In [117]:
import sqlalchemy as sqla

In [118]:
db = sqla.create_engine("sqlite:////Users/kumak/pydata/mydata.sqlite")

In [119]:
pd.read_sql("select * from test", db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
